In [ ]:
Test03_到每個網頁爬商品資料_Yahoo

In [1]:
import pandas as pd
import re, time, requests
from selenium import webdriver
from bs4 import BeautifulSoup

# 爬取點入網址

In [2]:
def FindLinks(url):
    linklist = []
    driver = webdriver.Chrome()
    driver.get(url)
    soup = BeautifulSoup(driver.page_source)

    links = soup.find_all('a', {'class':'BaseGridItem__content___3LORP'})
    linklist = []
    for link in links:
        k = link.get('href')
        linklist.append(k)
    return linklist

### 測試主畫面有無爬到各點入畫面網址

In [49]:
urls = 'https://tw.search.mall.yahoo.com/search/mall/product?kw=%E6%98%86%E7%9B%88&p=%E6%98%86%E7%9B%88&cid=0&clv=0'
FindLinks(urls)

['https://tw.mall.yahoo.com/item/p0904196550128',
 'https://tw.mall.yahoo.com/item/p0904108728569',
 'https://tw.mall.yahoo.com/item/p090415882600',
 'https://tw.mall.yahoo.com/item/p0904144506389',
 'https://tw.mall.yahoo.com/item/p0904108729320',
 'https://tw.mall.yahoo.com/item/p0904157320416',
 'https://tw.mall.yahoo.com/item/p049917840109',
 'https://tw.mall.yahoo.com/item/p054277396957',
 'https://tw.mall.yahoo.com/item/p0844121957574',
 'https://tw.mall.yahoo.com/item/p030984579080']

# 爬取點入分頁資料

In [4]:
def GetGoods(response):
    soup2 = BeautifulSoup(response.text)
    #url = soup2.find('link')['href']  #多餘的
    df = pd.DataFrame(data = [{'TITLE':soup2.find('span', attrs={'itemprop':'name'}).text,
                               'Price':soup2.find('span', attrs={'class':'has_promo_price'}), 
                               'Activity Price':soup2.find('span',attrs={'class':'price'}).text }],
                               columns = ['TITLE', 'Price', 'Activity Price']) 
    return df

### 測試點入分頁有無爬到

In [74]:
url = 'https://tw.mall.yahoo.com/item/GENIUS-%E6%98%86%E7%9B%88-DX-110-USB-%E7%86%B1%E5%8A%9B%E7%B4%85-%E6%9C%89%E7%B7%9A%E5%85%89%E5%AD%B8%E6%BB%91%E9%BC%A0-p0904196550128'
resp = requests.get(url)
GetGoods(resp)

,TITLE,Price,Activity Price
0,GENIUS 昆盈 DX-110 (USB) 熱力紅 有線光學滑鼠,149元,135元


In [7]:
url = 'https://tw.mall.yahoo.com/item/Logitech%E7%BE%85%E6%8A%80-%E7%84%A1%E7%B7%9A%E6%BB%91%E9%BC%A0M235-%E7%81%B0%E3%80%90%E6%84%9B%E8%B2%B7%E3%80%91-p067085113817'
resp = requests.get(url)
GetGoods(resp)

,TITLE,Price,Activity Price
0,Logitech羅技 無線滑鼠M235-灰【愛買】,None,399元


# 開始爬取

In [5]:
urls = 'https://tw.search.mall.yahoo.com/search/mall/product?cid=hp&clv=0&kw=logitech%E7%84%A1%E7%B7%9A%E6%BB%91%E9%BC%A0&p=logitech%E7%84%A1%E7%B7%9A%E6%BB%91%E9%BC%A0'
ndf = []
for i in FindLinks(urls):
    res = requests.get(i)
    g = GetGoods(res)
    ndf.append(g)
    time.sleep(1)
ndf

[                          TITLE Price Activity Price
 0  Logitech 羅技 M235 2.4G 無線滑鼠 灰  None           399元,
                                            TITLE Price Activity Price
 0  [富廉網] 羅技 Logitech MK220 無線鍵盤滑鼠組 (中文鍵盤) 限量下殺!!  None           499元,
                           TITLE Price Activity Price
 0  Logitech 羅技 MK235 無線鍵盤滑鼠組 中文  None           599元,
                          TITLE Price Activity Price
 0  Logitech羅技 靜音無線滑鼠M331-黑【愛買】  None           599元,
                                TITLE   Price Activity Price
 0  Logitech 羅技 M235 三色 Wireless 無線滑鼠  [399元]           399元,
                           TITLE   Price Activity Price
 0  Logitech 羅技 M350 鵝卵石無線滑鼠-石墨灰  [699元]           699元,
                                               TITLE    Price Activity Price
 0  Logitech 羅技 MX Master 3 無線藍牙多工滑鼠 黑色【送 羅技限量皮革電腦包】  [3990元]          3990元,
                         TITLE    Price Activity Price
 0  ［logitech 羅技］MK345 無線鍵盤滑鼠組  [1190元]          1130元,
                    TITLE   Pr

# 合併資料

In [6]:
df_all = pd.concat(ndf)
df_all

,TITLE,Price,Activity Price
0,Logitech 羅技 M235 2.4G 無線滑鼠 灰,None,399元
0,[富廉網] 羅技 Logitech MK220 無線鍵盤滑鼠組 (中文鍵盤) 限量下殺!!,None,499元
0,Logitech 羅技 MK235 無線鍵盤滑鼠組 中文,None,599元
0,Logitech羅技 靜音無線滑鼠M331-黑【愛買】,None,599元
0,Logitech 羅技 M235 三色 Wireless 無線滑鼠,[399元],399元
0,Logitech 羅技 M350 鵝卵石無線滑鼠-石墨灰,[699元],699元
0,Logitech 羅技 MX Master 3 無線藍牙多工滑鼠 黑色【送 羅技限量皮革電腦包】,[3990元],3990元
0,［logitech 羅技］MK345 無線鍵盤滑鼠組,[1190元],1130元
0,Logitech 羅技 B170 無線滑鼠,[299元],299元
0,Logitech 羅技 G304 無線遊戲滑鼠,None,939元
